In [118]:
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
def logistic(x, y, num_iteration=1000, learning_rate=1e-3):
    r, c = x.shape
    p = c + 1
    X = np.hstack((np.ones((r,1)), x))
    beta = 2*np.random.randn(p, 1)-1
    for i in range(num_iteration):
        pr = sigmoid(X.dot(beta))
        beta = beta + learning_rate* X.T.dot(y-pr)
    return beta
def sigmoid(x):
    return 1.0 / (1 + np.exp(-x))


dataset from https://www.kaggle.com/datasets/teejmahal20/airline-passenger-satisfaction?select=train.csv

In [108]:
import pandas as pd
import seaborn as sns
data=pd.read_csv("Data.csv")
data=pd.get_dummies(data, columns=["Gender","Customer Type","Type of Travel","Class",'satisfaction'], drop_first=True)
data.replace({True:1,False:0},inplace=True)
data = data.drop(columns=["id","Unnamed: 0"])
data=data.dropna()
data.head()

,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,...,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,Gender_Male,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Eco,Class_Eco Plus,satisfaction_satisfied
0,13,460,3,4,3,1,5,3,5,5,...,5,5,25,18.0,1,0,1,0,1,0
1,25,235,3,2,3,3,1,3,1,1,...,4,1,1,6.0,1,1,0,0,0,0
2,26,1142,2,2,2,2,5,5,5,5,...,4,5,0,0.0,0,0,0,0,0,1
3,25,562,2,5,5,5,2,2,2,2,...,4,2,11,9.0,0,0,0,0,0,0
4,61,214,3,3,3,3,4,5,5,3,...,3,3,0,0.0,1,0,0,0,0,1


In [109]:
X = data.iloc[:, :-1].to_numpy() 
y = data.iloc[:, -1].to_numpy().reshape(-1, 1) 

In [110]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,y,train_size=0.8,random_state=True)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(Xtrain)
X_test_scaled = scaler.transform(Xtest)

In [119]:
from sklearn.metrics import f1_score
def predict(x, beta, threshold=0.5):
    X = np.hstack((np.ones((x.shape[0], 1)), x))
    probabilities = sigmoid(X.dot(beta))
    return (probabilities >= threshold).astype(int)

logistic_beta = logistic(X_train_scaled, Ytrain)
Ypred = predict(X_test_scaled, logistic_beta)
accuracy = np.mean(Ypred == Ytest)
print(f"Accuracy on test data: {accuracy:.2f}")
print(f1_score(y_true=Ytest,y_pred=Ypred))

Accuracy on test data: 0.87
0.8450863138892022


In [120]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(max_iter=5000)
model.fit(X_train_scaled,Ytrain.ravel())
pred=model.predict(X_test_scaled)
print(np.mean(pred==Ytest.ravel()))
print(f1_score(y_true=Ytest.ravel(),y_pred=pred))

0.8737873449490805
0.8508186433909521
